## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mtinko,TZ,-4.5500,34.8500,59.32,98,100,4.92,overcast clouds
1,1,Jamtara,IN,23.9500,86.8000,58.19,86,1,9.31,clear sky
2,2,Lebu,CL,-37.6167,-73.6500,64.24,75,1,17.47,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,55.02,41,33,2.42,scattered clouds
4,4,Longyearbyen,SJ,78.2186,15.6401,15.64,67,0,20.71,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Rikitea,PF,-23.1203,-134.9692,78.46,71,9,10.63,clear sky
10,10,Lazaro Cardenas,MX,17.9583,-102.2000,81.25,55,2,7.56,clear sky
15,15,Touros,BR,-5.1989,-35.4608,81.50,59,26,15.19,scattered clouds
16,16,Atuona,PF,-9.8000,-139.0333,78.19,78,29,16.22,light rain
20,20,Vaini,TO,-21.2000,-175.2000,75.36,100,100,8.05,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                199
City                   199
Country                198
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Rikitea,PF,-23.1203,-134.9692,78.46,71,9,10.63,clear sky
10,10,Lazaro Cardenas,MX,17.9583,-102.2000,81.25,55,2,7.56,clear sky
15,15,Touros,BR,-5.1989,-35.4608,81.50,59,26,15.19,scattered clouds
16,16,Atuona,PF,-9.8000,-139.0333,78.19,78,29,16.22,light rain
20,20,Vaini,TO,-21.2000,-175.2000,75.36,100,100,8.05,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Rikitea,PF,78.46,clear sky,-23.1203,-134.9692,
10,Lazaro Cardenas,MX,81.25,clear sky,17.9583,-102.2000,
15,Touros,BR,81.50,scattered clouds,-5.1989,-35.4608,
16,Atuona,PF,78.19,light rain,-9.8000,-139.0333,
20,Vaini,TO,75.36,light rain,-21.2000,-175.2000,
26,Mahebourg,MU,80.26,light rain,-20.4081,57.7000,
28,Kieta,PG,81.37,overcast clouds,-6.2167,155.6333,
34,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,
35,Kahului,US,76.84,clear sky,20.8947,-156.4700,
39,Kavieng,PG,81.77,overcast clouds,-2.5744,150.7967,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
     #6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()
#Source: https://github.com/cmmgw/World_Weather_Analysis/blob/main/Vacation_Search/Vacation_Search.ipynb

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Rikitea,PF,78.46,clear sky,-23.1203,-134.9692,People ThankYou
10,Lazaro Cardenas,MX,81.25,clear sky,17.9583,-102.2000,quinta antigua
15,Touros,BR,81.50,scattered clouds,-5.1989,-35.4608,INN NEW HORIZON
16,Atuona,PF,78.19,light rain,-9.8000,-139.0333,Villa Enata
20,Vaini,TO,75.36,light rain,-21.2000,-175.2000,Keleti Beach Resort


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))